# Homework: Phrasal Chunking

TODO

In [14]:
import sys
import os
import perc
from bilstmcrf.util import *
from bilstmcrf.BiLSTM_CRF import BiLSTM_CRF, BiLSTM_Enc_Dec_CRF

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
tagset = perc.read_tagset("data/tagset.txt")

print("reading testing data ...", file=sys.stderr)
test_data = perc.read_labeled_data("data/dev.txt", "data/dev.feats", verbose=False)
print("done.", file=sys.stderr)

print('loading model...', file=sys.stderr)
model_data = load_model("h600layer2lr0.01enc.model")
word_idx = model_data['word_index']
speech_tag_idx = model_data['speech_tag_index']
tag2idx = model_data['tag_index']
idx2tag = model_data['reverse_tag_index']
layer = 2
hidden = 600
pos_dim = 64
model = BiLSTM_Enc_Dec_CRF(len(speech_tag_idx), len(tag2idx), device, layer, hidden, pos_dim)
model.load_state_dict(model_data['model'])
model.to(device)
print('done.', file=sys.stderr)

print('preparing testing data...', file=sys.stderr)
test_tuples = prepare_test_data(test_data, speech_tag_idx)
print('done.', file=sys.stderr)

predicted_tags = test_model(model, test_tuples, idx2tag, device)

output = format_prediction(predicted_tags, test_data)

reading testing data ...
done.
loading model...
done.
preparing testing data...
initializing ELMo embedding... 
loaded. 
100%|██████████| 500/500 [00:18<00:00, 26.47it/s]done.


In [15]:
import score_chunks
boundary = "-X-" # something to use as boundary between sentences
outside = "O" # tag used to mark the outside of any chunk
conlleval = False # use conlleval (should be False for most use cases)
numfeats = 2 # number of columns to consider as features, typically "word POStag"
(test, _) = score_chunks.readTestFile(output, boundary, outside, conlleval, numfeats)
with open("data/reference500.txt") as f:
    (reference, _) = score_chunks.readTestFile(f.read(), boundary, outside, conlleval, numfeats)
print("Score: %.2f" % score_chunks.corpus_fmeasure(reference, test, False))

processed 500 sentences with 10375 tokens and 5783 phrases; found phrases: 5809; correct phrases: 5564
             ADJP: precision:  78.00%; recall:  78.79%; F1:  78.39; found:    100; correct:     99
             ADVP: precision:  76.71%; recall:  83.17%; F1:  79.81; found:    219; correct:    202
            CONJP: precision: 100.00%; recall:  40.00%; F1:  57.14; found:      2; correct:      5
             INTJ: precision: 100.00%; recall: 100.00%; F1: 100.00; found:      1; correct:      1
               NP: precision:  97.12%; recall:  97.12%; F1:  97.12; found:   3026; correct:   3026
               PP: precision:  97.66%; recall:  98.94%; F1:  98.29; found:   1237; correct:   1221
              PRT: precision:  83.33%; recall:  68.18%; F1:  75.00; found:     18; correct:     22
             SBAR: precision:  89.81%; recall:  90.65%; F1:  90.23; found:    108; correct:    107
               VP: precision:  96.17%; recall:  96.00%; F1:  96.09; found:   1098; correct:   1100
accura